Package installations\

In [1]:
!pip install pandas pyodbc sqlalchemy

  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.7 MB 6.8 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.7 MB 4.5 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.7 MB 4.5 MB/s eta 0:00:02
   ------------- -------------------------- 3.4/9.7 MB 4.2 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.7 MB 4.1 MB/s eta 0:00:02
   --------------------- ------------------ 5.2/9.7 MB 4.1 MB/s eta 0:00:02
   ------------------------ --------------- 6.0/9.7 MB 4.1 MB/s eta 0:00:01
   -------------------------- ------------- 6.6/9.7 MB 4.0 MB/s eta 0:00:01
   ------------------------------- -------- 7.6/9.7 MB 4.0 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.7 MB 4.0 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.7 MB 4.0 MB/s eta 0:00:01
   -----------------


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Import Libraries

In [2]:
import pandas as pd
import json
import pyodbc
from sqlalchemy import create_engine

Load Customers JSON

In [3]:
with open("customers.json") as f:
    customers_data = json.load(f)

df_customers = pd.DataFrame(customers_data)

df_customers.head()

,CustomerID,FirstName,LastName,Gender,Region,SSN
0,C0001,Gregory,Miller,M,Ohho,082573197
1,C0002,Marvin,NaN,male,california,309-6256-21
2,C0003,Gregory,Smith,Male,New Yorkk,407071725
3,C0004,Edward,Davis,male,Ohho,948.18.5740
4,C0005,Reginald,Dawson,male,Texas,975.72.3056


Load Products CSV

In [4]:
df_products = pd.read_csv("products.csv")

df_products.head()

,ProductID,ProductName,Category
0,1000,Washing Machine,Home Appliances
1,1001,Refrigerator,Home Appliances
2,1002,Blender,Home Appliances
3,1003,Microwave,Home Appliances
4,1004,Vacuum Cleaner,Home Appliances


Load Sales CSV

In [5]:
df_sales = pd.read_csv("sales 1.csv")

df_sales.head()

,SaleID,ProductID,CustomerID,SalesAmount,Quantity,Timestamp
0,c0078cd1-3ddc-439a-b41d-7498e8e75e62,1020,C0460,1639.10,18.0,2024-08-09T20:21:00
1,be8532af-16b7-432c-a13d-4c6e6836d12d,1029,C0198,186.11,42.0,2024-11-28T05:01:00
2,8666f277-cc58-4fde-9d58-697da9e040f2,1012,C0524,1712.72,49.0,2024-10-13T00:36:00
3,383e0a2b-7e00-454e-a269-b2f73ecd33f2,1016,C0285,NaN,50.0,2024-07-06T23:39:00
4,d9a7f974-9572-48dd-aa36-3f96b280dfc7,1021,C0369,66.36,21.0,2024-06-08T19:58:00


Handle Missing Values

In [6]:
df_customers.info()
df_customers.isnull().sum()

<class 'pandas.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   CustomerID  1100 non-null   str  
 1   FirstName   1100 non-null   str  
 2   LastName    780 non-null    str  
 3   Gender      1100 non-null   str  
 4   Region      1031 non-null   str  
 5   SSN         1100 non-null   str  
dtypes: str(6)
memory usage: 51.7 KB


CustomerID      0
FirstName       0
LastName      320
Gender          0
Region         69
SSN             0
dtype: int64

✅ Handle Missing Values

In [16]:
df_customers['FirstName'] = df_customers['FirstName'].fillna("Unknown")
df_customers['LastName'] = df_customers['LastName'].fillna("Unknown")
df_customers['LastName'] = df_customers['LastName'].astype('string')
df_customers.dropna(subset=['CustomerID'], inplace=True)

Normalize Text

In [17]:
df_customers['Gender'] = df_customers['Gender'].str.lower().str.strip()
df_customers['Region'] = df_customers['Region'].str.lower().str.strip()
df_customers['Region'] = df_customers['Region'].fillna('Unassigned')

final transformation

In [29]:
df_customers['LastName'] = df_customers['LastName'].fillna('Unknown')
df_customers['Region'] = df_customers['Region'].fillna('Unassigned')

df_customers['FirstName'] = df_customers['FirstName'].str.title()
df_customers['LastName'] = df_customers['LastName'].str.title()
df_customers['Region'] = df_customers['Region'].str.title()
df_customers['Gender'] = df_customers['Gender'].str.capitalize()

#df_customers.drop(columns=['FullName'], inplace=True)

#df_customers = df_customers.drop(columns=['SSN'])

Transform Products Data

In [9]:
df_products.info()
df_products.isnull().sum()

<class 'pandas.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   ProductID    50 non-null     int64
 1   ProductName  50 non-null     str  
 2   Category     50 non-null     str  
dtypes: int64(1), str(2)
memory usage: 1.3 KB


ProductID      0
ProductName    0
Category       0
dtype: int64

Fix Issues

In [10]:
df_products.dropna(subset=['ProductID'], inplace=True)

df_products['Category'] = df_products['Category'].str.lower().str.strip()
df_products['ProductName'] = df_products['ProductName'].str.strip()

Transform Sales Data

In [11]:
df_sales.info()
df_sales.isnull().sum()

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SaleID       10000 non-null  str    
 1   ProductID    10000 non-null  int64  
 2   CustomerID   10000 non-null  str    
 3   SalesAmount  9503 non-null   float64
 4   Quantity     9887 non-null   float64
 5   Timestamp    10000 non-null  str    
dtypes: float64(2), int64(1), str(3)
memory usage: 468.9 KB


SaleID           0
ProductID        0
CustomerID       0
SalesAmount    497
Quantity       113
Timestamp        0
dtype: int64

Fix Data Types

In [13]:
df_sales['Timestamp'] = pd.to_datetime(df_sales['Timestamp'])

df_sales['SalesAmount'] = df_sales['SalesAmount'].astype(float)
df_sales['Quantity'] = df_sales['Quantity'].astype('Int64')

Remove Critical Missing Rows

In [14]:
df_sales.dropna(subset=['SaleID', 'CustomerID', 'ProductID'], inplace=True)

LOAD PHASE → Connect Python → SQL Server

In [21]:
from sqlalchemy import create_engine

server = 'localhost'
database = 'RetailDW'

connection_string = (
    "mssql+pyodbc://@localhost/RetailDW"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

engine = create_engine(connection_string)

In [25]:
pd.read_sql("SELECT TOP 5 * FROM dbo.DimCustomer", engine)

,CustomerID,FirstName,LastName,Gender,Region


Load Dimension Tables FIRST

In [35]:
df_customers.head()

,CustomerID,FirstName,LastName,Gender,Region
0,1,Gregory,Miller,M,Ohho
1,2,Marvin,Unknown,Male,California
2,3,Gregory,Smith,Male,New Yorkk
3,4,Edward,Davis,Male,Ohho
4,5,Reginald,Dawson,Male,Texas


coverting customer id in the format C0043 to 43 as int

In [33]:
df_customers['CustomerID'] = df_customers['CustomerID'].str.replace('C', '').astype(int)

In [34]:
df_sales['CustomerID'] = df_sales['CustomerID'].str.replace('C', '').astype(int)

In [43]:
df_customers['CustomerID'].duplicated().sum()

np.int64(0)

In [42]:
df_customers = df_customers.drop_duplicates(subset=['CustomerID'])

load dimCustomer

In [46]:
df_customers.to_sql(
    name='DimCustomer',
    con=engine,
    if_exists='append',
    index=False
)

162

Load DimProduct

In [47]:
df_products.to_sql(
    name='DimProduct',
    con=engine,
    if_exists='append',
    index=False
)

50

Load Fact Table LAST
Load FactSales

In [51]:
df_sales.to_sql(
    name='FactSales',
    con=engine,
    if_exists='append',
    index=False
)

228